In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import simplejson as json
pd.set_option('display.max_rows', 1000) 
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
#purpose: create our dataframe

#default column headers
col_headers = ['Name', 'Department', 'Title', 'Regular', 'Retro', 'Other', 'Overtime', 'Injury', 'Detail', 'Quinn', 'Total', 'Postal']

#create an empty df with default values
earnings_df = pd.DataFrame(columns = col_headers)

#add column 'Year' and set to NaN
earnings_df['Year'] = np.nan

#there are 9 sepearate CSV files, representing employee earnings reports for 9 years
for year in range(2011, 2020):
    
    #read in the CSV for the given year, set it to variable next_df
    next_df = pd.read_csv('./data/salaries_' + str(year) + '.csv', skiprows=1, names=col_headers, encoding='latin1')
    
    #add the column 'Year' to next_df and set to the given year
    next_df['Year'] = year
    
    #in the 2013 and 2014 datasets, title and department columns are in the wrong order
    if year == 2013 or year == 2014:
        
        #get the list of columns (including year)
        col_list = list(next_df)
        
        #swap the order of title and department in the list
        col_list[1], col_list[2] = col_list[2], col_list[1]
        
        #set the dataframe's columns to the new list
        next_df.columns = col_list
    
    #add next_df to earnings_df
    earnings_df = pd.concat([earnings_df, next_df], sort=False)

In [3]:
earnings_df.sample(500)

,Name,Department,Title,Regular,Retro,Other,Overtime,Injury,Detail,Quinn,Total,Postal,Year
4563,"Da Rosa,Katia F.",BPS Orchard Gardens K-8 Pilot,Teacher,$73227.25,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$73227.25,02126,2015.00
18973,"Shum,Cindy L",Boston Public Schools,Teacher,$82463.94,$744.78,$0.00,$0.00,$0.00,$0.00,$0.00,$83208.72,02038-2594,2013.00
11050,"MacKenzie,Jeremy James",BPS Lee Elementary,Teacher,"106,091.19","2,678.67",NaN,NaN,NaN,NaN,NaN,"108,769.86",02122,2018.00
8901,"Balti,Aadina A.",Boston Public Schools,Teacher,$92991.70,$0.00,$4742.00,$0.00,$0.00,$0.00,$0.00,$97733.70,02143-2801,2011.00
237,"Watson,Denise K",Boston Police Department,Police Officer,"103,303.31",NaN,"15,581.02","4,348.74",NaN,184.00,NaN,"123,417.07",02121,2018.00
1534,"Blevins,Michael",Traffic Division,Vehicle Impound Specialist ##,$42795.71,$0.00,$251.75,$24168.05,$0.00,$0.00,$0.00,$67215.51,02122,2015.00
20426,"Tierney,Michelle M",Boston Public Schools,Teacher,$61823.59,$558.47,$445.00,$0.00,$0.00,$0.00,$0.00,$62827.06,02184-5718,2013.00
7594,"Cela,Laura",Dpt of Innovation & Technology,Sr Data Proc Sys Anl,"$97,647.06",NaN,"$3,674.85",$609.14,NaN,NaN,NaN,"$101,931.05",2459,2017.00
18476,"Shaller,Catherine",Boston Public Schools,Substitute Teacher,$56940.50,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$56940.50,01450-1008,2011.00
10491,"Collins,Randy Matthew",Boston Public Schools,Teacher,$54473.71,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$54473.71,02050-3575,2011.00


In [4]:
#purpose: initial cleaning for our dataframe

#targeted (hard-coded) cleaning for specific rows
earnings_df = earnings_df.loc[earnings_df['Department']!='DEPARTMENT_NAME']

#dimensions are qualitative columns, facts are quantitative columns
facts = ['Regular', 'Retro', 'Other', 'Overtime', 'Injury', 'Detail', 'Quinn', 'Total']
    
##cast year to type 'int'
earnings_df['Year'] = earnings_df['Year'].astype(int)

#clean the facts columns and convert from type 'object' to 'float'
earnings_df[facts] = earnings_df[facts].astype(str).applymap(lambda x: x.strip())
earnings_df[facts] = earnings_df[facts].replace({'^-$|^None$|^nan$|\)':0, ',':'', '\$':'', ' ':'', '^\(':'-'}, regex=True)
earnings_df[facts] = earnings_df[facts].astype(float)

#remove the Detail and Quinn columns and add them to the Other column
earnings_df['Other'] = earnings_df[['Other', 'Detail', 'Quinn']].sum(axis=1)
earnings_df = earnings_df[['Name', 'Department', 'Title', 'Year', 'Regular', 'Retro', 'Overtime', 'Injury', 'Other', 'Total', 'Postal']]

In [5]:
earnings_df.head(500)

,Name,Department,Title,Year,Regular,Retro,Overtime,Injury,Other,Total,Postal
0,"Abadi,Kidani A",Assessing Department,Property Officer (Asn),2011,33065.38,0.00,379.49,0.00,0.00,33444.87,02118-3126
1,"Ablon,Jordan N",ASD Office Of Labor Relation,Asst Corp Counsel III,2011,76051.24,0.00,0.00,0.00,1321.03,77372.27,02135-5943
2,"Accardi,Patricia",Transportation-Parking Clerk,Chief Claims Investigator,2011,56430.79,0.00,418.58,0.00,0.00,56849.37,02081-3751
3,"Ackerly,Lyn E.",Boston Public Library,Spec Library Asst I,2011,35058.78,0.00,439.19,0.00,0.00,35497.97,02118-0000
4,"Adams,Carey L.",Law Department,Prin Clerk,2011,41588.83,0.00,0.00,0.00,0.00,41588.83,02131-4834
5,"Adams,Dean",Public Works Department,Highway Maint Frprs (Pwd)##,2011,37693.81,0.00,11485.32,0.00,6761.54,55940.67,02124-1112
6,"Adams,Natasha",Boston Cntr - Youth & Families,Youth Worker,2011,38330.84,0.00,47.14,0.00,0.00,38377.98,02124-4418
7,"Adario,Anthony J",ASD Human Resources,Supvising Claims Agent (Asd),2011,85214.42,0.00,0.00,0.00,1629.36,86843.78,02132-3000
8,"Addessa,Rocco",Property Management,Jr Building Custodian,2011,39322.79,0.00,3720.81,0.00,0.00,43043.60,02128-1617
9,"Afonseca,Jose",Boston Cntr - Youth & Families,Certified Seasonal Lifeguard,2011,4400.45,0.00,137.99,0.00,0.00,4538.44,02124-1214


In [6]:
#convert dimensions from type 'object' to 'string' and remove leading/trailing whitespace
dimensions = ['Name', 'Department', 'Title', 'Postal']
earnings_df[dimensions] = earnings_df[dimensions].astype(str)

In [7]:
#purpose: clean Postal column

#add a 0 to the front of any code with 4 digits
mask = earnings_df['Postal'].str.len() == 4
earnings_df.loc[mask, 'Postal'] = '0' + earnings_df.loc[mask, 'Postal']

#remove delivery route number from any codes that have it (number after hyphen)
earnings_df['Postal'] = earnings_df['Postal'].str.split('-', expand=True)[0]

#any postal codes with non-numeric characters will be set to UNKNOWN
earnings_df.loc[earnings_df['Postal'].str.match('[A-Z]', na=False), 'Postal'] = 'UNKNOWN'

#hard cleaning for specific rows
earnings_df.loc[earnings_df['Name'].str.match('Ostiguy,David M'), 'Postal'] = '02327'
earnings_df.loc[earnings_df['Name'].str.match('Karales,George Alfred'), 'Postal'] = '02170'
earnings_df.loc[earnings_df['Name'].str.match('Smith,Kenneth J'), 'Postal'] = '02124'
earnings_df.loc[earnings_df['Name'].str.match('Thomas,Sarita J'), 'Postal'] = '02125'
earnings_df.loc[earnings_df['Name'].str.match('Morris,Judith A.'), 'Postal'] = '02170'
earnings_df.loc[earnings_df['Name'].str.match('Mendez,Jose R'), 'Postal'] = '02135'
earnings_df.loc[earnings_df['Name'].str.match('Morrison,June'), 'Postal'] = '02481'

In [8]:
#purpose: clean Name column
earnings_df['Name'] = earnings_df['Name'].replace({'\.':''}, regex=True).str.upper()

#split name column into two
name = earnings_df['Name'].str.split(',', n = 1, expand = True)
earnings_df['Last']=name[0]
earnings_df['First']=name[1]
earnings_df.drop(columns =["Name"], inplace = True) 

dimensions = ['First', 'Last', 'Department', 'Title', 'Postal']

In [9]:
#return list of all unique values for a given column sorted in alphabetical order
def show_unique(column):
    optns = earnings_df[column].unique()
    optns.sort()
    return optns

#purpose: get pivot table of employee counts by department
#rec is an optional column that specifies what record to filter on, set to everything by default
def count_by_group(column, rec=r'(.*?)'):
    
    #create dataframe with department, year, and counts
    group_counts = earnings_df.groupby([column, 'Year'])['Total'].count().reset_index(name="count")
    
    #remove BPS schools from result
    #dept_counts = dept_counts.loc[dept_counts['Department'].str.contains(BPS_str, regex=True) == False]
    
    #create the pivot table table, with calculated sums for each row and column
    group_counts_table = pd.pivot_table(group_counts, values='count', index=column, columns='Year', aggfunc='sum', fill_value=0, margins=True).reset_index()
    
    #get rid of the calculated sums by row, doesn't make sense for time series data
    return group_counts_table.iloc[:,0:-1].loc[group_counts_table[column].str.match(rec)]

In [10]:
#purpose: preliminary cleaning for Title column - create uniform spacing, convert to all caps, and remove unwanted characters
earnings_df['Title'] = earnings_df['Title'].replace({'\.':'', '(?<=[a-z])([A-Z])':r' \1', '\(':' (', '\)':') ', '\\\\':'', '&':' AND ', '\,':'/', '\#':'', '\s+':' '}, regex=True).str.upper()
earnings_df[dimensions] = earnings_df[dimensions].applymap(lambda x: x.strip())

In [11]:
#find all titles with given phrase
def find_matching_titles(s):
    titles = pd.DataFrame(show_unique('Title'), columns=["Title"])
    return titles.loc[titles["Title"].str.contains('(^|[\/\s\(])(' + s + ')([\/\s\)]|$)')]
    
find_matching_titles('JOURNEYPRS')

c:\users\cpsch\gitrepos\bostonsalaries\venv\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,Title
1175,MAINT MECH PLUMBER (JOURNEYPRS)


In [12]:
#purpose: preliminary cleaning for Title column to unstick words that were stuck together
earnings_df['Title']=earnings_df['Title'].replace({'([A-Z])(OF)([\/\s\)]|$)':r'\1 OF ', '([A-Z])(BPD)':r'\1 \2', '(BPD)([A-Z])':r'\1 \2'},regex=True)

In [13]:
#purpose: replace abbreviations and spelling errors with proper words in title column

#CHALLENGES --> 
#OP/OPER: OPERATOR VS OPERATIONS
#SP/SPEC: SPECIAL VS SPECIALIST
#SERV/SVC: SERVICE VS SERVICES
#COM/COMM: COMMUNITY VS COMMUNICATIONS VS COMMISSIONER VS COMMISSION VS COMMITTEE
abbrevs = {
    'ADMIN AND FINANCE':'ADMINISTRATION AND FINANCE',
    'ADMIN/FINANCE':'ADMINISTRATION AND FINANCE',
    'OFFC':'OFFICER',
    'OFFCR':'OFFICER',
    'SEN':'SENIOR',
    'DET':'DETECTIVE',
    'SUP':'SUPERVISOR',
    'SUPV':'SUPERVISOR',
    'SPV':'SUPERVISOR',
    'EXEC':'EXECUTIVE',
    'AN':'ANALYST',
    'ANL':'ANALYST',
    'ANAL':'ANALYST',
    'TECH':'TECHNICIAN',
    'DEPT':'DEPARTMENT',
    'EQUIP':'EQUIPMENT',
    'EQUIPMENT OPER':'EQUIPMENT OPERATOR',
    'ALARM OPER':'ALARM OPERATOR',
    'METER OPER':'METER OPERATOR',
    'COMPUTER OPER':'COMPUTER OPERATOR',
    'PROJ':'PROJECT',
    'SP':'SPECIAL',
    'STFF':'STAFF',
    'ACAD':'ACADEMY',
    'INSTR':'INSTRUCTOR',
    'ASST':'ASSISTANT',
    'ASSIST':'ASSISTANT',
    'ASS':'ASSISTANT',
    'DEP':'DEPUTY',
    'SUPN':'SUPERINTENDENT',
    'SYS':'SYSTEMS',
    'COOR':'COORDINATOR',
    'COORD':'COORDINATOR',
    'SEC':'SECRETARY',
    'LIEUT':'LIEUTENANT',
    'LT':'LIEUTENANT',
    'MAINT':'MAINTENANCE',
    'MAIN':'MAINTENANCE',
    'DIR':'DIRECTOR',
    'MGMT':'MANAGEMENT',
    'MGR':'MANAGER',
    'MNGR':'MANAGER',
    'MANGR':'MANAGER',
    'MED':'MEDICAL',
    'PROC':'PROCESSING',
    'CORP':'CORPORATION',
    'ASSOC':'ASSOCIATE',
    'ASSESS OPER MANAGEMENT':'ASSESSING OPERATIONS MANAGEMENT',
    'OPER':'OPERATIONS',
    'OP':'OPERATOR',
    'INC COMM':'INCIDENT COMMAND',
    'COMM SERV':'COMMUNITY SERVICE',
    'COMM OUTREACH':'COMMUNITY OUTREACH',
    'ASSISTANT COMM':'ASSISTANT COMMISSIONER',
    'DEPUTY COMM':'DEPUTY COMMISSIONER',
    'COMM OFFICE':'COMMISSIONERS OFFICE',
    'RADIO COMM':'RADIO COMMUNICATIONS',
    'COMM EQUIPMENT':'COMMUNICATIONS EQUIPMENT',
    'COMMUNIC':'COMMUNICATIONS',
    'COMMUN':'COMMUNICATIONS',
    'COMM LEADER':'COMMUNITY LEADER',
    'FIRE COMM':'FIRE COMMISSIONER',
    'HOUSING COMM':'HOUSING COMMISSION',
    'SCHOOL COMM':'SCHOOL COMMITTEE',
    'COMM AND INTERG':'COMMUNITY AND INTERGOVERNMENTAL',
    'BLDG':'BUILDINGS',
    'BLDGS':'BUILDINGS',
    'BDG':'BUILDINGS',
    'REG':'REGISTRAR',
    'SERV':'SERVICE',
    'SVC':'SERVICE',
    'SRV':'SERVICE',
    'PRIN':'PRINCIPAL',
    'PARA':'PARAPROFESSIONAL',
    'DIST':'DISTRICT',
    'FF':'FIRE FIGHTER',
    'A AND F':'ADMINISTRATION AND FINANCE',
    'F':'FIRE',
    'INSTRUC':'INSTRUCTOR',
    'SR':'SENIOR',
    'JR':'JUNIOR',
    'MECH':'MECHANIC',
    'MECHA':'MECHANIC',
    'MACH':'MACHINE',
    'GEN':'GENERAL',
    'ADMN':'ADMIN',
    'ADM':'ADMIN',
    'ENG':'ENGINEER',
    'STRUCT':'STRUCTURAL',
    'FRPRS':'FOREPERSON',
    'FRPR':'FOREPERSON',
    'FOREPRS':'FOREPERSON',
    'CONST':'CONSTRUCTION',
    'LBR':'LABORER',
    'RPR':'REPAIR',
    'REP':'REPAIR',
    'SPEC':'SPECIAL',
    'INCT':'INCIDENT',
    'COMND':'COMMAND',
    'FIN COM':'FINANCE COMMISSION',
    'MAS':'MASTER',
    'RPPRS':'REPAIRPERSON',
    'REPPRS':'REPAIRPERSON',
    'REPRPRS':'REPAIRPERSON',
    'REPAIRPR':'REPAIRPERSON',
    'REPAIRPRS':'REPAIRPERSON',
    'RPPR':'REPAIR PERSON',
    'WKG':'WORKING',
    'PW':'PUBLIC WORKS',
    'P W':'PUBLIC WORKS',
    'HVY':'HEAVY',
    'MTR':'MOTOR',
    'INSP':'INSPECTOR',
    'INSPEC':'INSPECTOR',
    'TRA':'TRAFFIC',
    'OPR':'OPERATIONS',
    'MEO':'MOTOR EQUIPMENT OPERATOR',
    'CFM':'(CFM)',
    'ELEC':'ELECTRIC',
    'EQUI':'EQUIPMENT',
    'COLL TRS':'COLLECTOR TREASURER',
    'COLL-TRS':'COLLECTOR TREASURER',
    'ACNTNG':'ACCOUNTING',
    'CRFTSPRS':'CRAFTSPERSON',
    'COMMSS':'COMMISSIONER',
    'COMR':'COMMISSIONER',
    'COMMIS':'COMMISSIONER',
    'CFM':'',
    'COUNSLR':'COUNSELOR',
    'BD':'BOARD',
    'MEMBER BOARD':'MEMBER OF BOARD',
    'LIB':'LIBRARIAN',
    'LIBR':'LIBRARIAN',
    'LIBRARIN':'LIBRARIAN',
    'SVCS':'SERVICES',
    'CAMP JO':'(CAMP JOY)',
    'CAM JO':'(CAMP JOY)',
    'SER':'SERVICES',
    'PROT':'PROTECTIVE',
    'REL':'RELATIONS',
    'SUPVISING':'SUPERVISING',
    'PROP':'PROPERTY',
    'DISP':'DISPATCHER',
    'CHF':'CHIEF',
    'PMDGRAFF REMOVAL':'(GRAFFITI REMOVAL)',
    'PAINT':'PAINTER',
    'ENFORCE':'ENFORCEMENT',
    'DEVELOP':'DEVELOPMENT',
    'DEVEL':'DEVELOPMENT',
    'PROG':'PROGRAM',
    'PWD':'',
    'SWIM':'SWIMMING',
    'REGNL':'REGIONAL',
    'ACCTNG':'ACCOUNTING',
    'ACCT':'ACCOUNTING',
    'ENGR':'ENGINEER',
    'EQU':'EQUIPMENT',
    'EQ':'EQUIPMENT',
    'ANIM CNTL':'ANIMAL CONTROL',
    'OFCR':'OFFICER',
    'CLRK':'CLERK',
    'PARKS AND REC':'PARKS AND RECREATION',
    'P AND R':'PARKS AND RECREATION',
    '\(PARK\)':'(PARKS AND RECREATION)',
    'SPC':'SPECIAL',
    'HDQ':'HEADQUARTER',
    'TRANS':'TRANSPORTATION',
    'DISPCH':'DISPATCHER',
    'SUB':'SUBSTITUTE',
    'CUST':'CUSTODIAN',
    'NEIGH':'NEIGHBORHOOD',
    'YTH':'YOUTH',
    'HE':'',
    'BE':'',
    'FGR PRT EV':'FINGERPRINT EVIDENCE',
    'CH':'CHIEF',
    'SUM SCH':'SUMMER SCHOOL',
    'COM SCH':'COMMUNITY SCHOOL',
    'FAM':'FAMILY',
    'BPDFLEET':'BPD FLEET',
    'IBPDFLEET':'I (BPD FLEET',
    'IIBPDFLEET':'II (BPD FLEET',
    'EVIDENC':'EVIDENCE',
    'TECHNCN':'TECHNICIAN',
    'TCH':'TECHNICIAN',
    'SP ED':'SPECIAL ED',
    'SPED':'SPECIAL ED',
    'DP':'DATA PROCESSING',
    'PREV':'PREVENTION',
    'SCUBA DIV':'SCUBA DIVER',
    'DIV':'DIVISION',
    'POL':'POLICE',
    'OFFR':'OFFICER',
    'CLASSIFICATN':'CLASSIFICATION',
    'ATTN':'ATTENDANT',
    'CAFE':'CAFETERIA',
    'SCHL':'SCHOOL',
    'SCH':'SCHOOL',
    'HOSP':'HOSPITAL',
    'ASSTO':'ASSISTANT TO',
    'CHIEFOF':'CHIEF OF',
    'EXC':"EXECUTIVE",
    'OFF':'OFFICE',
    'SRGT':'SERGEANT',
    'RET':'RETIREMENT',
    'RETIRE':'RETIREMENT',
    'RETIREME':'RETIREMENT',
    'RECYCLE':'RECYCLING',
    'REAS':'RESEARCH',
    'INVESTNS':'INVESTIGATIONS',
    'INVEST':'INVESTIGATOR',
    'HACKNEY':'HACKNEY UNIT',
    'TOTHE':'TO THE',
    'PRGS':'PROGRAMS',
    'JOURNEYPRS':'JOURNEYPERSON',
    'BLD':'BUILDINGS',
    'PLMG AND GSFTG':'PLUMBING AND GASFITTING',
    'PLG AND GAS FTNG':'PLUMBING AND GASFITTING',
    'EL IN AND MNT':'ELECTRICAL INSPECTION AND MAINTENANCE',
    'AL':'ALTERATION',
    'ALT':'ALTERATION',
    'CLK':'CLERK',
    'ADV':'ADVANCED',
    'MAST\)':'MASTER',
    'ADR':'(RETIRED - ADR)',
    'LIBRRIN':'LIBRARIAN',
    'SPECL':'SPECIALIST',
    'LAB RELATIONS':'LABOR RELATIONS',
    'C AND REPAIR':'CONSTRUCTION AND REPAIR',
    'CONSTR':'CONSTRUCTION',
    'PMDCONST':'PMD CONSTRUCTION',
    'AUD':'AUDITOR',
    'ENVRNMNTL':'ENVIRONMENTAL',
    'PR OF':'PARAPROFESSIONAL',
    'HOT LINE':'HOTLINE',
    'FCOMMISSIONER':'FIRE COMMISSIONER',
    'PROFESS':'PROFESSIONAL'
}

#to prevent words within words from accidentally being changed, lets ensure that values from above dictionary must be
#between certain characters to change
for key in abbrevs:
    abbrevs[key] = ' '+abbrevs[key]+' '
    new_key = '(^|[\s\(\)\-\/])'+key+'([\s\(\)\-\/]|$)'
    earnings_df['Title'] = earnings_df['Title'].replace(new_key, abbrevs[key], regex=True).str.strip()

earnings_df['Title'] = earnings_df['Title'].replace({'SPECIAL$':'SPECIALIST'}, regex=True)

In [14]:
count_by_group('Title').sort_values(2019, ascending=False).head(1000)

Year,Title,2011,2012,2013,2014,2015,2016,2017,2018,2019
2003,All,20509,21140,22469,22233,21902,22046,22245,23603,23312
1906,TEACHER,5121,5249,5512,5426,5375,5363,5437,5541,5255
1265,PARAPROFESSIONAL,1108,1161,1209,1113,1180,1187,1215,1416,1356
1311,POLICE OFFICER,1281,1231,1308,1349,1273,1279,1331,1334,1346
847,FIRE FIGHTER,804,816,861,873,815,837,795,792,820
1832,SUBSTITUTE TEACHER,770,743,1045,837,793,771,759,983,724
1278,PART-TIME CAFETERIA ATTENDANT,360,389,417,360,356,341,334,375,374
1132,LUNCH HOUR MONITORS,388,368,424,384,377,370,360,392,341
277,CAB MONITOR,252,319,379,343,354,365,306,232,337
1303,POLICE DETECTIVE,281,292,290,286,286,330,298,296,309


In [15]:
show_unique('Department')

array(['ASD Graphic Arts', 'ASD Human Resources',
       'ASD Intergvernmtl Relations', 'ASD Office Of Labor Relation',
       'ASD Office of Budget Mangmnt', 'ASD Purchasing Division',
       'Accountability', 'Achievement Gap', 'Administration and Finance',
       'Advancement & Ext. Affairs', 'Age Strong',
       'Alighieri Montessori School', 'Arts & Cultural Development',
       'Assessing Department', 'Asst Superintendent-Network A',
       'Asst Superintendent-Network B', 'Asst Superintendent-Network C',
       'Asst Superintendent-Network D', 'Asst Superintendent-Network E',
       'Asst Superintendent-Network F', 'Asst Superintendent-Network G',
       'Auditing Department', 'BPS Adams Elementary',
       'BPS Adult Education', 'BPS Alternative Education',
       'BPS Alternative Education HS', 'BPS Another Course To Colleg',
       'BPS Athletics', 'BPS Bates Elementary',
       'BPS Beethoven Elementary', 'BPS Blackstone Elementary',
       'BPS Boston Arts Academy', 'BPS Bo

In [16]:
find_matching_titles('LAB')

c:\users\cpsch\gitrepos\bostonsalaries\venv\lib\site-packages\pandas\core\strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,Title
930,FIU DIGITAL LAB SUPERVISOR
1090,LAB INFORMATIOIN MANAGEMENT ADMIN BPD
1091,LAB TECHNICIAN


In [17]:
#purpose: preliminary cleaning for Department column - create uniform spacing, convert to all caps, and remove unwanted characters

earnings_df['Department'] = earnings_df['Department'].replace({'\.':'', '(?<=[a-z])([A-Z])':r' \1', '\/':' AND ', '\\\\':'', '&':' AND ', '\,':' ', '\#':''}, regex=True).str.upper()
earnings_df['Department'] = earnings_df['Department'].replace('\s+', ' ', regex=True).str.strip()

In [18]:
#purpose: most depts are BPS related, lets move them to new 'Program' column and set dept to 'Boston Public Schools'

BPS_str = '^BPS|ELEMENTARY|ACADEMY|K-8|MIDDLE|HIGH|SCHOOL|ACAD$|PILOT| EEC$| ELC$| EES$|9-12|ACHIEVEMENT GAP|STUDENT|SUPERINTENDENT|CHIEF ACADEMIC OFFICER|ENGLISH LANGUAGE LEARN|ACCOUNTABILITY|^ADVANCEMENT|ENROLLMENT|^EARLY LEARNING|^HPEC|^INFO AND INSTR|TEACHING|CAREER AND TECHNICAL ED|CHIEF OF STAFF|CHIEF FINANCIAL OFFICER|CHIEF OPERATING OFFICER|^COMMUNICATIONS$|FOOD AND NUTRITION SVC|INNOVATION DEPARTMENT|INSTITUTIONAL ADVANCEMT|LEGAL ADVISOR|PROFESSIONAL DEVELOPMNT|RESEARCH ASSESS AND EVAL|STRATEGY DEPARTMENT'
earnings_df.loc[earnings_df['Department'].str.contains(BPS_str, regex=True),'Program'] = earnings_df.loc[earnings_df['Department'].str.contains(BPS_str, regex=True),'Department']
earnings_df.loc[earnings_df['Department'].str.contains(BPS_str, regex=True),'Department'] = 'BOSTON PUBLIC SCHOOLS'

In [19]:
show_unique('Department')

array(['ADMINISTRATION AND FINANCE', 'AGE STRONG',
       'ARTS AND CULTURAL DEVELOPMENT', 'ASD GRAPHIC ARTS',
       'ASD HUMAN RESOURCES', 'ASD INTERGVERNMTL RELATIONS',
       'ASD OFFICE OF BUDGET MANGMNT', 'ASD OFFICE OF LABOR RELATION',
       'ASD PURCHASING DIVISION', 'ASSESSING DEPARTMENT',
       'AUDITING DEPARTMENT', 'BOSTON CITY COUNCIL',
       'BOSTON CNTR - YOUTH AND FAMILIES',
       'BOSTON CNTR-YOUTH AND FAMILIES', 'BOSTON FIRE DEPARTMENT',
       'BOSTON POLICE DEPARTMENT', 'BOSTON PUBLIC LIBRARY',
       'BOSTON PUBLIC SCHOOLS', 'BOSTON RETIREMENT SYSTEM',
       'CEMETERY DIVISION', 'CITY CLERK', 'COMM FOR PERSONS WITH DISABIL',
       'CONSUMER AFFAIRS AND LICENSING', 'DEPT OF VOTER MOBILIZATION',
       'DIPLOMAS PLUS', 'DND NEIGHBORHOOD DEVELOPMENT',
       'DPT OF INNOVATION AND TECHNOLOGY', 'ELDERLY COMMISSION',
       'ELECTION DIVISION', 'EMERGENCY MANAGEMENT',
       'ENVIRONMENT DEPARTMENT', 'FAIR HOUSING AND EQUITY',
       'FINANCE COMMISSION', 'HBI RET

In [20]:
#purpose: the operating budget report in the Boston Open Data Portal has much better data on the organization of Boston's govt
#lets get all the cabinets, departments, and programs from the operating budget

budget_df = pd.read_csv('./data/operating_budget.csv', encoding='latin1')
budget_df = budget_df.replace(np.nan,'')
budget_df = budget_df.drop_duplicates(['Cabinet','Dept','Program'])[['Cabinet','Dept','Program']].apply(lambda x: x.str.upper())
budget_df = budget_df.replace({'\.':' ', '\&':' AND ', '\,':' ', '\s+':' '}, regex=True).apply(lambda x: x.str.strip())

#some minor corrections to the data
budget_df = budget_df.replace({'LIBRARY DEPARTMENT':'BOSTON PUBLIC LIBRARY', 
                               'COMMISSION FOR PERSONS W/DISABILITIES':'COMMISSION FOR PERSONS WITH DISABILITIES', 
                               'LICENSING_BOARD':'LICENSING BOARD',
                               'BOSTON VETS':"VETERANS' SERVICES"})

In [21]:
budget_df

,Cabinet,Dept,Program
0,MAYORS CABINET,MAYOR'S OFFICE,MAYOR'S ADMINISTRATION
5,MAYORS CABINET,MAYOR'S OFFICE,MAYOR'S EXECUTIVE
7,MAYORS CABINET,MAYOR'S OFFICE,MAYOR'S POLICY AND PLANNING
11,MAYORS CABINET,MAYOR'S OFFICE,NEW URBAN MECHANICS
16,MAYORS CABINET,MAYOR'S OFFICE,MAYOR'S COMMUNICATIONS
20,MAYORS CABINET,ELECTION DEPARTMENT,ELECTION ADMINISTRATION
25,MAYORS CABINET,ELECTION DEPARTMENT,VOTER REGISTRATION
28,MAYORS CABINET,ELECTION DEPARTMENT,ELECTION ACTIVITIES
32,MAYORS CABINET,ELECTION DEPARTMENT,ANNUAL LISTING
35,MAYORS CABINET,INTERGOVERNMENTAL RELATIONS,IGR


In [22]:
#purpose: replace incorrect department names with correct ones when applicable

dept_abbrevs = {
    '^OFFICE OF FINANCE AND BUDGET$':'ADMINISTRATION AND FINANCE', #*OFFICE OF FINANCE AND BUDGET: 2014 - 2015
    '^OFFICE OF ADMIN AND FINANCE$':'ADMINISTRATION AND FINANCE', 
    '^DND NEIGHBORHOOD DEVELOPMENT$':'NEIGHBORHOOD DEVELOPMENT',
    '^ARTS AND CULTURAL DEVELOPMENT$':'OFFICE OF ARTS AND CULTURE', 
    '^ASD HUMAN RESOURCES$':'HUMAN RESOURCES',
    '^ASD INTERGVERNMTL RELATIONS$':'INTERGOVERNMENTAL RELATIONS',
    '^ASD OFFICE OF BUDGET MANGMNT$':'BUDGET MANAGEMENT',
    '^ASD OFFICE OF LABOR RELATION$':'OFFICE OF LABOR RELATIONS',
    '^ASD PURCHASING DIVISION$':'PROCUREMENT', #*AKA PURCHASING
    '^BOSTON CITY COUNCIL$':'CITY COUNCIL',
    '^BOSTON CNTR \- YOUTH AND FAMILIES$':'BOSTON CENTER FOR YOUTH AND FAMILIES',
    '^BOSTON CNTR\-YOUTH AND FAMILIES$':'BOSTON CENTER FOR YOUTH AND FAMILIES',
    '^BOSTON FIRE DEPARTMENT$':'FIRE DEPARTMENT',
    '^BOSTON POLICE DEPARTMENT$':'POLICE DEPARTMENT',
    '^BOSTON RETIREMENT SYSTEM$':'RETIREMENT DEPARTMENT',
    '^STATE BOSTON RETIREMENT SYST$':'RETIREMENT DEPARTMENT',
    '^COMM FOR PERSONS WITH DISABIL$':'COMMISSION FOR PERSONS WITH DISABILITIES',
    '^DEPT OF VOTER MOBILIZATION$':'ELECTION DEPARTMENT', #*DEPARTMENT OF VOTER MOBILIZATION: <2013
    '^DPT OF INNOVATION AND TECHNOLOGY$':'DEPARTMENT OF INNOVATION AND TECHNOLOGY',
    '^ELDERLY COMMISSION$':'AGE STRONG', #*ELDERLY COMMISSION <2018
    '^ELECTION DIVISION$':'ELECTION DEPARTMENT',
    '^IMMIGRANT ADVANCEMENT$':'OFFICE FOR IMMIGRANT ADVANCEMENT',
    '^INSPECTIONAL SERVICES DEPT$':'INSPECTIONAL SERVICES DEPARTMENT',
    '^OFC CHF PUBLIC WORKS TRANSPORT$':'OFFICE OF STREETS', #*OFFICE OF CHIEF OF PUBLIC WORKS AND TRANSPORT <2013
    '^OFC OF STRTS TRNSP AND SANI$':'OFFICE OF STREETS', #*OFFICE OF STREETS, TRANSPORTATION, AND SANITATION 2014 - 2015
    '^OFFICE OF CIVIL RIGHTS$':'FAIR HOUSING AND EQUITY', #*OFFICE OF CIVIL RIGHTS <2013
    '^OFFICE OF NEW BOSTONIANS$':'OFFICE FOR IMMIGRANT ADVANCEMENT', #*OFFICE OF NEW BOSTONIANS <2017
    '^PARKS DEPARTMENT$':'PARKS AND RECREATION DEPARTMENT',
    '^PROPERTY MANAGEMENT$':'PROPERTY MANAGEMENT DEPARTMENT',
    '^YOUTH FUND$':'YOUTH ENGAGEMENT AND EMPLOYMENT', #*YOUTH FUND <2013
    "^WOMEN'S COMMISSION$":"WOMEN'S ADVANCEMENT" #*WOMEN'S COMMISSION <2013
}

earnings_df['Department'] = earnings_df['Department'].replace(dept_abbrevs, regex=True)

In [23]:
show_unique('Department')

array(['ADMINISTRATION AND FINANCE', 'AGE STRONG', 'ASD GRAPHIC ARTS',
       'ASSESSING DEPARTMENT', 'AUDITING DEPARTMENT',
       'BOSTON CENTER FOR YOUTH AND FAMILIES', 'BOSTON PUBLIC LIBRARY',
       'BOSTON PUBLIC SCHOOLS', 'BUDGET MANAGEMENT', 'CEMETERY DIVISION',
       'CITY CLERK', 'CITY COUNCIL',
       'COMMISSION FOR PERSONS WITH DISABILITIES',
       'CONSUMER AFFAIRS AND LICENSING',
       'DEPARTMENT OF INNOVATION AND TECHNOLOGY', 'DIPLOMAS PLUS',
       'ELECTION DEPARTMENT', 'EMERGENCY MANAGEMENT',
       'ENVIRONMENT DEPARTMENT', 'FAIR HOUSING AND EQUITY',
       'FINANCE COMMISSION', 'FIRE DEPARTMENT', 'HBI RETIREES ET AL',
       'HUMAN RESOURCES', 'INSPECTIONAL SERVICES DEPARTMENT',
       'INTERGOVERNMENTAL RELATIONS', 'LAW DEPARTMENT', 'LICENSING BOARD',
       "MAYOR'S OFFICE", "MAYOR'S OFFICE-PUBLIC INFO",
       'NEIGHBORHOOD DEVELOPMENT', 'NEIGHBORHOOD SERVICES',
       'OFC BOSTON RESIDENTS JOB POL', 'OFFICE FOR IMMIGRANT ADVANCEMENT',
       'OFFICE OF ARTS

In [24]:
#purpose: some depts should actually be programs, this function can be used to set departments to programs

def replace_dept(orig_dept, new_dept, new_prog):
    earnings_df.loc[earnings_df['Department'].str.contains(orig_dept, regex=True),'Program'] = new_prog
    earnings_df.loc[earnings_df['Department'].str.contains(orig_dept, regex=True),'Department'] = new_dept

In [25]:
#use the above function to fix some of the Department data
replace_dept('CEMETERY DIVISION', 'PARKS AND RECREATION DEPARTMENT', 'CEMETERY')
replace_dept('LICENSING BOARD', 'CONSUMER AFFAIRS AND LICENSING', 'LICESNING BOARD')
replace_dept("MAYOR\'S OFFICE\-PUBLIC INFO", "MAYOR'S OFFICE", "MAYOR'S COMMUNICATIONS")
replace_dept('OFC BOSTON RESIDENTS JOB POL', 'OFFICE OF ECONOMIC DEVELOPMENT', 'BOSTON RESIDENTS JOB POLICY OFFICE')
replace_dept('SMALL AND LOCAL BUSINESS', 'OFFICE OF ECONOMIC DEVELOPMENT', 'SMALL AND LOCAL BUSINESS')
replace_dept('TRAFFIC DIVISION', 'TRANSPORTATION DEPARTMENT', 'TRAFFIC DIVISION')
replace_dept('TRANSPORTATION-PARKING CLERK', 'TRANSPORTATION DEPARTMENT', 'PARKING CLERK')
replace_dept('TREASURY-COLLECTING DIVISION', 'TREASURY DEPARTMENT', 'TREASURY DIVISION')
replace_dept('TREASURY-TREASURY DIVISION', 'TREASURY DEPARTMENT', 'COLLECTING DIVISION')
replace_dept('WORKERS COMPENSATION SERVICE', 'HUMAN RESOURCES', 'WORKERS COMP')
replace_dept('HBI RETIREES ET AL', 'HUMAN RESOURCES', 'HEALTH BENEFITS AND INSURANCE')

In [26]:
show_unique('Department')

array(['ADMINISTRATION AND FINANCE', 'AGE STRONG', 'ASD GRAPHIC ARTS',
       'ASSESSING DEPARTMENT', 'AUDITING DEPARTMENT',
       'BOSTON CENTER FOR YOUTH AND FAMILIES', 'BOSTON PUBLIC LIBRARY',
       'BOSTON PUBLIC SCHOOLS', 'BUDGET MANAGEMENT', 'CITY CLERK',
       'CITY COUNCIL', 'COMMISSION FOR PERSONS WITH DISABILITIES',
       'CONSUMER AFFAIRS AND LICENSING',
       'DEPARTMENT OF INNOVATION AND TECHNOLOGY', 'DIPLOMAS PLUS',
       'ELECTION DEPARTMENT', 'EMERGENCY MANAGEMENT',
       'ENVIRONMENT DEPARTMENT', 'FAIR HOUSING AND EQUITY',
       'FINANCE COMMISSION', 'FIRE DEPARTMENT', 'HUMAN RESOURCES',
       'INSPECTIONAL SERVICES DEPARTMENT', 'INTERGOVERNMENTAL RELATIONS',
       'LAW DEPARTMENT', "MAYOR'S OFFICE", 'NEIGHBORHOOD DEVELOPMENT',
       'NEIGHBORHOOD SERVICES', 'OFFICE FOR IMMIGRANT ADVANCEMENT',
       'OFFICE OF ARTS AND CULTURE', 'OFFICE OF ECONOMIC DEVELOPMENT',
       'OFFICE OF LABOR RELATIONS', 'OFFICE OF STREETS',
       'OFFICE OF TOURISM', 'PARKS AND 

In [27]:
#preliminary cleaning for program column
earnings_df['Program'] = earnings_df['Program'].replace({'^BPS':'', 'BOSTON PUBLIC SCHOOLS':'', '\"':'', '\-NETWORK':' - NETWORK'}, regex=True).str.strip()
earnings_df['Program'] = earnings_df['Program'].replace(np.nan, '')
show_unique('Program')

array(['', 'ACCOUNTABILITY', 'ACHIEVEMENT GAP', 'ADAMS ELEMENTARY',
       'ADULT EDUCATION', 'ADVANCEMENT AND EXT AFFAIRS',
       'ALIGHIERI MONTESSORI SCHOOL', 'ALTERNATIVE EDUCATION',
       'ALTERNATIVE EDUCATION HS', 'ANOTHER COURSE TO COLLEG',
       'ASST SUPERINTENDENT - NETWORK A',
       'ASST SUPERINTENDENT - NETWORK B',
       'ASST SUPERINTENDENT - NETWORK C',
       'ASST SUPERINTENDENT - NETWORK D',
       'ASST SUPERINTENDENT - NETWORK E',
       'ASST SUPERINTENDENT - NETWORK F',
       'ASST SUPERINTENDENT - NETWORK G', 'ATHLETICS', 'BALDWIN ELC',
       'BATES ELEMENTARY', 'BEETHOVEN ELEMENTARY',
       'BLACKSTONE ELEMENTARY', 'BOSTON ARTS ACADEMY',
       'BOSTON COLLABORATIVE HIGH SCH', 'BOSTON COMM LEADERSHIP AC',
       'BOSTON EVENING ACADEMY', 'BOSTON INTERNATIONAL HS',
       'BOSTON LATIN', 'BOSTON MIDDLE SCHOOL ACADEMY',
       'BOSTON RESIDENTS JOB POLICY OFFICE', 'BOSTON SCHOOL COMMITTEE',
       'BRADLEY ELEMENTARY', 'BRIGHTON HIGH', 'BTU PILOT', 'BURKE

In [28]:
prog_abbrevs = {
    'MGMT':' MANAGEMENT ',
    'EXT AFFAIRS':' EXTERNAL AFFAIRS ',
    'CL10':' ',
    'COM ACD':' COMMUNITY ACADEMY OF ',
    'P A SHAW':' PA SHAW ',
    'NURS':' NURSES ',
    'WEST ROXBURY HIGH':' WEST ROXBURY ACADEMY ', 
    'WREC\:':' ',
    'HPEC\:':' ',
    'MC CORMACK MIDDLE':' MCCORMACK MIDDLE ', 
    'KENNEDY EM':' EDWARD M KENNEDY ',
    'KENNEDY JF':' JOHN F KENNEDY ',
    'KENNEDY PJ':' PATRICK J KENNEDY ',
    'GREENWOOD S':' SARAH GREENWOOD ',
    'MC KINLEY MIDDLE':' MCKINLEY MIDDLE ',
    'HORACE MANN':'HORACE MANN SCHOOL',
    'WITHTHROP':' WINTHROP ',
    'SOUTH BOSTON HS - EXCEL':' EXCEL HIGH SCHOOL ',
    'MPHCOMMERCE':' MADISON PARK HIGH SCHOOL - COMMERCE ',
    'MPHCRAFTS':' MADISON PARK HIGH SCHOOL - CRAFTS ',
    'MPHHEALTH':' MADISON PARK HIGH SCHOOL - HEALTH ',
    'MPHFRESHMAN':' MADISON PARK HIGH SCHOOL - FRESHMAN ',
    'FACILITY MANAGEMENT AND A AND R':' FACILITIES MANAGEMENT ',
    'FACILITY MANAGEMENT':' FACILITIES MANAGEMENT ',
    'FACILITITES MANAGEMENT':' FACILITIES MANAGEMENT ',
    'GREENWOOD E':' ELIHU GREENWOOD ',
    'ALTERNATIVE EDUCATION HIGH SCHOOL':' ALTERNATIVE EDUCATION ',
    'COLLEG':' COLLEGE ',
    'SCH':' SCHOOL ',  
    'INTERVTN CT':' INTERVENTION CENTER ',
    'COUSELING':' COUNSELING ',
    'SERVICE':' SERVICES ',
    'SER':' SERVICES ',
    'HI':' HIGH ',
    'ED':' EDUCATION ',
    'HS':' HIGH SCHOOL ',
    'ACAD':' ACADEMY ',
    'SVC':' SERVICES ',
    'ASST':' ASSISTANT ',
    'COMM':' COMMUNITY ',
    'AC':' ACADEMY ',
    'ST':' STREET ',
    'HEARING AND APPEAL AND ATTENDANC':' HEARINGS AND APPEALS ',
    'KITCHEN FOOD':' KITCHEN AND FOOD ',
    'LEARN':' LEARNING ',
    'FAM AND STUDENT ENGAGEMT':' FAMILY AND STUDENT ENGAGEMENT ',
    'ADVANCEMT':' ADVANCEMENT ',
    'ACCOUNTABILITY':' TURNAROUND AND TRANSFORMATION ',
    'RESEARCH ASSESS AND EVAL':' DATA AND ACCOUNTABILITY '
}

for key in prog_abbrevs:
    new_key = '(^|[\s\(\)\-])'+key+'([\s\(\)\-]|$)'
    earnings_df['Program'] = earnings_df['Program'].replace(new_key, prog_abbrevs[key], regex=True).str.strip()
    
earnings_df['Program'] = earnings_df['Program'] .replace({'ELEMENTARY$':'ELEMENTARY SCHOOL',
                                                          'MIDDLE$':'MIDDLE SCHOOL',
                                                          'HIGH$':'HIGH SCHOOL',
                                                          'PILOT$':'PILOT SCHOOL'}, regex=True)

In [29]:
show_unique('Program')

array(['', 'ACHIEVEMENT GAP', 'ADAMS ELEMENTARY SCHOOL',
       'ADULT EDUCATION', 'ADVANCEMENT AND EXTERNAL AFFAIRS',
       'ALIGHIERI MONTESSORI SCHOOL', 'ALTERNATIVE EDUCATION',
       'ALTERNATIVE EDUCATION HIGH SCHOOL', 'ANOTHER COURSE TO COLLEGE',
       'ASSISTANT SUPERINTENDENT - NETWORK A',
       'ASSISTANT SUPERINTENDENT - NETWORK B',
       'ASSISTANT SUPERINTENDENT - NETWORK C',
       'ASSISTANT SUPERINTENDENT - NETWORK D',
       'ASSISTANT SUPERINTENDENT - NETWORK E',
       'ASSISTANT SUPERINTENDENT - NETWORK F',
       'ASSISTANT SUPERINTENDENT - NETWORK G', 'ATHLETICS', 'BALDWIN ELC',
       'BATES ELEMENTARY SCHOOL', 'BEETHOVEN ELEMENTARY SCHOOL',
       'BLACKSTONE ELEMENTARY SCHOOL', 'BOSTON ARTS ACADEMY',
       'BOSTON COLLABORATIVE HIGH SCHOOL',
       'BOSTON COMMUNITY LEADERSHIP ACADEMY', 'BOSTON EVENING ACADEMY',
       'BOSTON INTERNATIONAL HIGH SCHOOL', 'BOSTON LATIN',
       'BOSTON MIDDLE SCHOOL ACADEMY',
       'BOSTON RESIDENTS JOB POLICY OFFICE', 'BO

In [30]:
earnings_df.sample(500)

,Department,Title,Year,Regular,Retro,Overtime,Injury,Other,Total,Postal,Last,First,Program
20503,BOSTON PUBLIC SCHOOLS,PRINCIPAL ELEMENTARY,2011,4488.42,0.00,0.00,0.00,0.00,4488.42,02129,ZRIKE JR,STEPHEN K,
14319,BOSTON PUBLIC SCHOOLS,PARAPROFESSIONAL,2013,27025.97,276.41,0.00,0.00,400.00,27702.38,02155,MITCHELL,MARCELL,
1538,POLICE DEPARTMENT,POLICE DETECTIVE,2015,74730.97,0.00,33850.74,0.00,72427.47,181009.18,02126,BLOCKER,AARON T,
9788,OFFICE OF TOURISM,PROJECT ASSISTANT,2016,7210.00,0.00,0.00,0.00,0.00,7210.00,02124,JACKSON,RASHAD L,
3530,PUBLIC WORKS DEPARTMENT,HEAVY MOTOR EQUIPMENT REPAIRPERSON CLASS III,2014,41479.40,0.00,4645.32,0.00,58.50,46183.22,02131,VALERA,MANUEL JESUS,
4769,BOSTON PUBLIC SCHOOLS,TEACHER,2018,105529.08,2846.84,0.00,0.00,0.00,108375.92,02121,BUDD,KARYSSA H,MCKINLEY MIDDLE SCHOOL
5081,BOSTON PUBLIC SCHOOLS,TEACHER,2014,67855.70,0.00,0.00,0.00,5507.80,73363.50,02130,MACLEOD-BLUVER,CAITLIN,BOSTON INTERNATIONAL HIGH SCHOOL
2374,POLICE DEPARTMENT,POLICE OFFICER,2018,24825.64,0.00,4240.71,69351.36,800.00,99217.71,02136,GAUVIN,JAMES A,
3711,POLICE DEPARTMENT,POLICE DETECTIVE,2011,75302.49,0.00,38496.98,0.00,24377.78,138177.25,02062,ACLOQUE,JEAN MOISE,
5684,BOSTON PUBLIC SCHOOLS,TEACHER,2013,95958.79,895.48,0.00,0.00,2250.00,99104.27,02184,DONOVAN,AMY M,


In [31]:
earnings_df.size

2592967

In [32]:
budget_df = budget_df[['Cabinet','Dept']].drop_duplicates()
earnings_df = earnings_df.join(budget_df.set_index('Dept'),how='left', on='Department')
earnings_df['Cabinet'] = earnings_df['Cabinet'].replace(np.nan, '')
earnings_df = earnings_df[['First','Last','Year','Cabinet','Department','Program','Title','Regular','Retro','Overtime','Injury','Other','Total','Postal']]

In [33]:
earnings_df = earnings_df.replace('',np.nan)

In [34]:
earnings_df.sort_values(['Last', 'First', 'Year']).head(500)

,First,Last,Year,Cabinet,Department,Program,Title,Regular,Retro,Overtime,Injury,Other,Total,Postal
19168,BRIAN,A'HEARN,2018,HEALTH AND HUMAN SERVICES,BOSTON CENTER FOR YOUTH AND FAMILIES,NaN,BUILDING ASSISTANT,24326.46,0.00,1230.40,0.00,0.00,25556.86,02132
16596,BRIAN,A'HEARN,2019,HEALTH AND HUMAN SERVICES,BOSTON CENTER FOR YOUTH AND FAMILIES,NaN,BUILDING ASSISTANT,37034.48,0.00,0.00,0.00,0.00,37034.48,02132
8362,RUBY A,ABABIO-FERNANDEZ,2011,EDUCATION,BOSTON PUBLIC SCHOOLS,NaN,PRINCIPAL ELEMENTARY,119578.40,0.00,0.00,0.00,0.00,119578.40,02301
14675,RUBY A,ABABIO-FERNANDEZ,2012,EDUCATION,BOSTON PUBLIC SCHOOLS,NaN,PRINCIPAL ELEMENTARY,87161.71,0.00,0.00,0.00,12122.88,99284.59,11001
0,RUBY A,ABABIO-FERNANDEZ,2013,EDUCATION,BOSTON PUBLIC SCHOOLS,NaN,PRINCIPAL ELEMENTARY,0.00,0.00,0.00,0.00,12122.88,12122.88,11001
0,KIDANI A,ABADI,2011,ADMINISTRATION AND FINANCE,ASSESSING DEPARTMENT,NaN,PROPERTY OFFICER (ASN),33065.38,0.00,379.49,0.00,0.00,33444.87,02118
240,KIDANI A,ABADI,2012,ADMINISTRATION AND FINANCE,ASSESSING DEPARTMENT,NaN,PROPERTY OFFICER (ASN),34554.41,157.84,0.00,0.00,918.06,35630.31,02118
1,KIDANI A,ABADI,2013,ADMINISTRATION AND FINANCE,ASSESSING DEPARTMENT,NaN,PROPERTY OFFICER,37588.30,0.00,0.00,0.00,431.87,38020.17,02118
122,KIDANI A,ABADI,2014,ADMINISTRATION AND FINANCE,ASSESSING DEPARTMENT,NaN,PROPERTY OFFICER,40303.47,0.00,0.00,0.00,50.00,40353.47,02118
0,KIDANI A,ABADI,2015,ADMINISTRATION AND FINANCE,ASSESSING DEPARTMENT,NaN,PROPERTY OFFICER,44006.26,0.00,324.12,0.00,275.00,44605.38,02118


In [40]:
#purpose: convert the dataframe into JSON, with one document for each employee that contains their first name, last name,
#and employment/earnings data by year

#group the df by first and last name
'''j = (earnings_df.groupby(['First','Last'], as_index=False)
             #add all the rest of the data into a dictionary
             .apply(lambda x: x.iloc[:,2:].to_dict(orient='records'))
             .reset_index()
             #rename the key with all the list of dictionaries to Employment
             .rename(columns={0:'Employment'})
             #convert to JSON
             .to_json(orient='records'))'''
j = earnings_df.sort_values(['Last','First','Year']).to_json(orient='records')

In [41]:
json.loads(j)

[{'First': 'BRIAN',
  'Last': "A'HEARN",
  'Year': 2018,
  'Cabinet': 'HEALTH AND HUMAN SERVICES',
  'Department': 'BOSTON CENTER FOR YOUTH AND FAMILIES',
  'Program': None,
  'Title': 'BUILDING ASSISTANT',
  'Regular': 24326.46,
  'Retro': 0.0,
  'Overtime': 1230.4,
  'Injury': 0.0,
  'Other': 0.0,
  'Total': 25556.86,
  'Postal': '02132'},
 {'First': 'BRIAN',
  'Last': "A'HEARN",
  'Year': 2019,
  'Cabinet': 'HEALTH AND HUMAN SERVICES',
  'Department': 'BOSTON CENTER FOR YOUTH AND FAMILIES',
  'Program': None,
  'Title': 'BUILDING ASSISTANT',
  'Regular': 37034.48,
  'Retro': 0.0,
  'Overtime': 0.0,
  'Injury': 0.0,
  'Other': 0.0,
  'Total': 37034.48,
  'Postal': '02132'},
 {'First': 'RUBY A',
  'Last': 'ABABIO-FERNANDEZ',
  'Year': 2011,
  'Cabinet': 'EDUCATION',
  'Department': 'BOSTON PUBLIC SCHOOLS',
  'Program': None,
  'Title': 'PRINCIPAL ELEMENTARY',
  'Regular': 119578.4,
  'Retro': 0.0,
  'Overtime': 0.0,
  'Injury': 0.0,
  'Other': 0.0,
  'Total': 119578.4,
  'Postal': '02

In [42]:
#purpose: remove all null fields from the JSON

#removes fields with null values from a dictionary
def remove_nulls(d):
    return {k: v for k, v in d.items() if v is not None}

#remove all null fields from JSON data
j = json.loads(j, object_hook=remove_nulls)

In [47]:
#connect to the database
client = MongoClient()
db = client['boston_salaries']
collection = db['salaries']

#clear the database
collection.delete_many({})

#insert the json data
collection.insert_many(j)

In [48]:
for val in collection.find({'$and':[{'First':'MARTIN J','Last':'WALSH'}]}):
    print(val)

{'_id': ObjectId('5efbb51d498609b27b75098d'), 'First': 'MARTIN J', 'Last': 'WALSH', 'Year': 2014, 'Cabinet': 'MAYORS CABINET', 'Department': "MAYOR'S OFFICE", 'Title': 'MAYOR', 'Regular': 164903.87, 'Retro': 0.0, 'Overtime': 0.0, 'Injury': 0.0, 'Other': 0.0, 'Total': 164903.87, 'Postal': '02125'}
{'_id': ObjectId('5efbb51d498609b27b75098e'), 'First': 'MARTIN J', 'Last': 'WALSH', 'Year': 2015, 'Cabinet': 'MAYORS CABINET', 'Department': "MAYOR'S OFFICE", 'Title': 'MAYOR', 'Regular': 181730.79, 'Retro': 0.0, 'Overtime': 0.0, 'Injury': 0.0, 'Other': 0.0, 'Total': 181730.79, 'Postal': '02125'}
{'_id': ObjectId('5efbb51d498609b27b75098f'), 'First': 'MARTIN J', 'Last': 'WALSH', 'Year': 2016, 'Cabinet': 'MAYORS CABINET', 'Department': "MAYOR'S OFFICE", 'Title': 'MAYOR', 'Regular': 175000.02, 'Retro': 0.0, 'Overtime': 0.0, 'Injury': 0.0, 'Other': 0.0, 'Total': 175000.02, 'Postal': '02124'}
{'_id': ObjectId('5efbb51d498609b27b750990'), 'First': 'MARTIN J', 'Last': 'WALSH', 'Year': 2017, 'Cabinet